In [1]:
import json
import re 
import copy
from importlib import reload

In [2]:
import sys
import clingo
import clingo.symbol as symbol

In [3]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")


thisdir = /home/luke/git/external/asp_recipe_graphs/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/asp_recipe_graphs/notebooks', '/home/luke/git/external/asp_recipe_graphs', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [4]:
## local modules
import asp_recipe_graphs.api.modules
reload(asp_recipe_graphs.api.modules)

from asp_recipe_graphs.api.modules import RE_FIND_TERMS
from asp_recipe_graphs.api.modules import RE_IS_DEFINITION
from asp_recipe_graphs.api.modules import ASP_MODULE_DEFINES
from asp_recipe_graphs.api.modules import ASP_MODULE_USES
from asp_recipe_graphs.api.modules import DEPENDENCY_MAP
from asp_recipe_graphs.api.modules import ALL_DEFINED
from asp_recipe_graphs.api.modules import ALL_USES
from asp_recipe_graphs.api.modules import UNDEFINED_TERMS
from asp_recipe_graphs.api.modules import get_dependencies
from asp_recipe_graphs.api.modules import what_defines
from asp_recipe_graphs.api.modules import what_uses

In [5]:
print(f"asp_module_defines = {ASP_MODULE_DEFINES}")
print()
print("Uses")
for m, uses in ASP_MODULE_USES.items():
    print(f"\t{m} uses {uses}")
print()
print(f"undefined_terms = {UNDEFINED_TERMS}")
print()

asp_module_defines = {'acceptability': {'acceptable_recipe'}, 'composition': {'composable', 'incompatible_node_types', 'empty_intersection'}, 'equivalence': {'equivalent'}, 'granularity': {'mapped_to_by_g_function', 'finer_grained', 'path_or_equal', 'more_specific', 'not_same_recipe'}, 'graph_properties': {'is_node', 'cyclic', 'has_outgoing_arc', 'has_incoming_arc', 'graph', 'path', 'connected_in'}, 'in_out_aligned': {'in_out_aligned'}, 'isomorphisms': {'isomorphic'}, 'recipe_graphs': {'given_recipe_graph', 'atomic_recipe_graph', 'recipe_graph', 'subrecipe_graph_of', 'high_indexes_allowed', 'graph', 'recipe', 'strict_subrecipe_graph'}, 'subrecipes': {'subrecipe_of', 'subrecipe_graph_of', 'strict_subrecipe_graph'}, 'type_hierarchies': {'same_type_path', 'subtype', 'subtype_or_equal', 'action_type', 'comestible_type'}}

Uses
	acceptability uses {'arc', 'type_of', 'acceptability_tuple', 'in', 'a', 'arcs', 'c', 'recipe'}
	composition uses {'out', 'same_type_path', 'recipe_graph', 'in', 'mi

In [6]:
RE_IS_DEFINITION = re.compile('^[a-z_]*\([a-zA-Z_,]*\) :-')
RE_IS_DEFINITION = re.compile('^([a-z][a-z_]*)\([a-zA-Z0-9_,]*\) :-')

line = 'subrecipe_of(RG1,TF1,RG2,TF2) :- '
#line = 'strict_subrecipe_graph(RG) :-              % need this for the '
RE_IS_DEFINITION.findall(line)


['subrecipe_of']

## Running ASP recipes

In [7]:

print(f"dependency_map = {DEPENDENCY_MAP}")

dependency_map = {'acceptability': {'recipe_graphs'}, 'composition': {'type_hierarchies', 'recipe_graphs'}, 'equivalence': {'isomorphisms', 'recipe_graphs'}, 'granularity': {'recipe_graphs', 'type_hierarchies', 'in_out_aligned', 'graph_properties', 'isomorphisms'}, 'graph_properties': set(), 'in_out_aligned': {'recipe_graphs'}, 'isomorphisms': {'recipe_graphs'}, 'recipe_graphs': {'type_hierarchies', 'graph_properties'}, 'subrecipes': {'recipe_graphs'}, 'type_hierarchies': set()}


In [8]:
specified = ['acceptability']
get_dependencies(specified)


to_load = ['acceptability']
specified = ['acceptability']
to_load = ['recipe_graphs']
specified = ['acceptability']
to_load = ['type_hierarchies', 'graph_properties']
specified = ['acceptability']
to_load = ['graph_properties']
specified = ['acceptability']


{'acceptability', 'graph_properties', 'recipe_graphs', 'type_hierarchies'}

In [9]:
what_defines('recipe', asp_module_defines)

NameError: name 'asp_module_defines' is not defined

In [ ]:
what_uses('recipe', asp_module_uses)

In [ ]:
'recipe' in asp_module_uses['acceptability']

## Older code

In [ ]:
graph = 'atomic_recipe_graph(rg2) atomic_recipe_graph(rg3) rg3_arc((a(1),"pour onto toast"),(c(4),"beans on toast")) rg3_arc((c(0),"heated beans"),(a(1),"pour onto toast")) rg3_arc((c(3),"buttered toast"),(a(1),"pour onto toast")) rg2_arc((a(0),"spread on toast"),(c(3),"buttered toast")) rg2_arc((c(1),"plain toast"),(a(0),"spread on toast")) rg2_arc((c(2),"butter"),(a(0),"spread on toast"))'

In [ ]:
def getatoms(graph):
    atoms = [e+")" for e in graph.split(") ")]
    atoms[-1] = atoms[-1][:-1]
    return atoms
    
        
atoms = getatoms(graph)

    
for x in atoms:
    print(x)



In [ ]:

def isarc(x):
    return "_arc" in x

def getsource(p):
    r = p[0]
    q = r.split("((")
    q2 = q[1]
    q3 = q2.split("(")
    q4 = q3[0]
    q5 = q3[1]
    q6 = q5[:-1]
    return q4+q6


def getsourcetxt(p):
    z = p[1]
    return z[:-1]


def gettarget(p):
    z = p[2]
    q2 = z.split("(")
    q3 = q2[1]
    q4 = q2[2]
    q5 = q4[:-1]
    return q3+q5


def gettargettxt(p):
    z = p[3]
    if z.endswith("))"):
        a = z[:-2]
    else:
        a = z[:-1]
    return a



def builddotstring(atoms):
    mystring = "digraph G {\n"
    for x in atoms:
        if isarc(x):
            p = x.split(",")
            #print(str(p))
            s1 = getsource(p)
            #print(s1)
            s2 = getsourcetxt(p)
            #print(s2)
            t1 = gettarget(p)
            #print(t1)
            t2 = gettargettxt(p)
            #print(t2)
            if "a" in s1:
                n1 = s1 + " [label= " + s2 + ", style=filled, color = yellow, shape=box];\n"
                n2 = t1 + " [label= " + t2 + ", style=filled, color = gray, shape=box];\n"
            else:
                n1 = s1 + " [label= " + s2 + ", style=filled, color = gray, shape=box];\n"
                n2 = t1 + " [label= " + t2 + ", style=filled, color = yellow, shape=box];\n"
            e = s1 + " -> " + t1
            mystring = mystring + n1 + n2 + e + ";\n"
    mystring = mystring + "}\n"
    print(mystring)


builddotstring(atoms)

In [ ]:
answer_sets_str = """
Answer: 1
atomic_recipe_graph(rg2) atomic_recipe_graph(rg3) rg3_arc((a(1),"pour onto toast"),(c(4),"beans on toast")) rg3_arc((c(0),"heated beans"),(a(1),"pour onto toast")) rg3_arc((c(3),"buttered toast"),(a(1),"pour onto toast")) rg2_arc((a(0),"spread on toast"),(c(3),"buttered toast")) rg2_arc((c(1),"plain toast"),(a(0),"spread on toast")) rg2_arc((c(2),"butter"),(a(0),"spread on toast"))
Answer: 2
atomic_recipe_graph(rg2) atomic_recipe_graph(rg3) rg3_arc((a(1),"pour onto toast"),(c(4),"beans on toast")) rg3_arc((c(0),"heated beans"),(a(1),"pour onto toast")) rg3_arc((c(3),"buttered toast"),(a(1),"pour onto toast")) rg2_arc((a(0),"spread on toast"),(c(3),"buttered toast")) rg2_arc((c(1),"plain toast"),(a(0),"spread on toast")) rg2_arc((c(2),"butter"),(a(0),"spread on toast"))
Answer: 3
atomic_recipe_graph(rg2) atomic_recipe_graph(rg3) rg3_arc((a(1),"pour onto toast"),(c(4),"beans on toast")) rg3_arc((c(0),"heated beans"),(a(1),"pour onto toast")) rg3_arc((c(3),"buttered toast"),(a(1),"pour onto toast")) rg2_arc((a(0),"spread on toast"),(c(3),"buttered toast")) rg2_arc((c(1),"plain toast"),(a(0),"spread on toast")) rg2_arc((c(2),"butter"),(a(0),"spread on toast"))
Answer: 4
atomic_recipe_graph(rg2) atomic_recipe_graph(rg3) rg3_arc((a(1),"pour onto toast"),(c(4),"beans on toast")) rg3_arc((c(0),"heated beans"),(a(1),"pour onto toast")) rg3_arc((c(3),"buttered toast"),(a(1),"pour onto toast")) rg2_arc((a(0),"spread on toast"),(c(3),"buttered toast")) rg2_arc((c(1),"plain toast"),(a(0),"spread on toast")) rg2_arc((c(2),"butter"),(a(0),"spread on toast"))
Answer: 5
atomic_recipe_graph(rg2) atomic_recipe_graph(rg3) rg3_arc((a(1),"pour onto toast"),(c(4),"beans on toast")) rg3_arc((c(0),"heated beans"),(a(1),"pour onto toast")) rg3_arc((c(3),"buttered toast"),(a(1),"pour onto toast")) rg2_arc((a(0),"spread on toast"),(c(3),"buttered toast")) rg2_arc((c(1),"plain toast"),(a(0),"spread on toast")) rg2_arc((c(2),"butter"),(a(0),"spread on toast"))
"""

digraphs = []
for line in answer_sets_str.split("\n"):
    if line.startswith("Answer:") or (len(line)==0):
        continue
    atoms = getatoms(line)
    digraph = builddotstring(atoms)
    digraphs.append(digraph)
digraphs

In [ ]:
import graphviz
dot = graphviz.Digraph()
dot.node('A', 'King Arthur')  # doctest: +NO_EXE
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')

dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')
dot

In [ ]:
help(dot.node)

In [ ]:
def builddot(atoms):
    dot = graphviz.Digraph()
    for x in atoms:
        if isarc(x):
            p = x.split(",")
            s1 = getsource(p)
            s2 = getsourcetxt(p)
            t1 = gettarget(p)
            t2 = gettargettxt(p)
            #print(t2)
            if "a" in s1:
                dot.node(s1, label=s2, style='filled', color='yellow', shape='box')
#                 n1 = s1 + " [label= " + s2 + ", style=filled, color = yellow, shape=box];\n"
                dot.node(t1, label=t2, style='filled', color='gray', shape='oval')
#                 n2 = t1 + " [label= " + t2 + ", style=filled, color = gray, shape=box];\n"
            else:
                dot.node(s1, label=s2, style='filled', color='gray', shape='oval')
#                 n1 = s1 + " [label= " + s2 + ", style=filled, color = gray, shape=box];\n"
                dot.node(t1, label=t2, style='filled', color='yellow', shape='box')
#                 n2 = t1 + " [label= " + t2 + ", style=filled, color = yellow, shape=box];\n"
#             e = s1 + " -> " + t1
            dot.edge(s1, t1)
#             mystring = mystring + n1 + n2 + e + ";\n"
#     mystring = mystring + "}\n"
    return dot


In [ ]:
dot = builddot(atoms)
dot

In [ ]:

lp_fnames = ["GraphRecipes.lp","ScratchExample.lp"]
lp_fnames = ["GraphRecipes.lp","SnacksOnToastExample.lp"]

In [ ]:
ctl = clingo.Control()
for lp_fname in lp_fnames:
    ctl.load(lp_fname)
ctl.ground([("base", [])])
#answer_sets = []
result = ctl.solve(on_model=lambda m: print("Answer: {}".format(m)))

In [ ]:
ctl = clingo.Control()
for lp_fname in lp_fnames:
    ctl.load(lp_fname)
ctl.ground([("base", [])])
models = []
with ctl.solve(yield_=True) as handle:
    for model in handle:
        models.append(model.symbols(shown=True))
models

In [ ]:
for atom in models[0]:
    print(f"atom.type = {atom.type}")
    if atom.type == symbol.SymbolType.Function:
        print(f"atom.name = {atom.name}")
        print(f"atom.arguments = {atom.arguments}")
        print(f"str(atom) = {str(atom)}")
        #print(f"atom.negative = {atom.negative}")
        #print(f"atom.number = {atom.number}")
        #print(f"atom.positive = {atom.positive}")
        #print(f"atom.string = {atom.string}")


In [ ]:
help(models[0][0])

In [ ]:
answer_sets[0].symbols(shown=True)

In [ ]:
help(answer_sets[0])

In [ ]:
help(result)

In [ ]:
help(ctl)

In [ ]:
# TO reproduce default clingo do this:
class Application:
    def __init__(self, name):
        self.program_name = name

    def main(self, ctl, files):
        if len(files) > 0:
            for f in files:
                ctl.load(f)
        else:
            ctl.load("-")
        ctl.solve()

        
        
clingo.clingo_main(Application(sys.argv[0]), sys.argv[1:])